In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import jieba
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# 设置matplotlib支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [2]:
def load_character_list(filename):
    """
    从文件中加载人物列表
    Args:
        filename: 人物列表文件名
    Returns:
        character_list: 人物列表
    """
    character_list = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            name = line.strip()
            if name:  # 跳过空行
                character_list.append(name)
    print(f"已加载 {len(character_list)} 个人物")
    return character_list

In [3]:
def add_characters_to_jieba(character_list):
    """
    将人物名字添加到jieba词典中
    Args:
        character_list: 人物列表
    """
    for char in character_list:
        jieba.add_word(char)
    print("已添加人物名字到jieba词典")

In [4]:
def read_and_tokenize_novels(novel_dir):
    """
    读取所有小说并分词
    Args:
        novel_dir: 小说文件所在目录
    Returns:
        sentences: 分词后的句子列表
    """
    sentences = []
    for filename in os.listdir(novel_dir):
        if filename.endswith('.txt'):
            print(f"正在处理文件: {filename}")
            with open(os.path.join(novel_dir, filename), 'r', encoding='utf-8') as f:
                text = f.read()
                # 按句子分割
                for line in text.split('\n'):
                    if line.strip():
                        words = list(jieba.cut(line.strip()))
                        sentences.append(words)
    print(f'处理完成，共处理了 {len(sentences)} 个句子')
    return sentences

In [5]:
def train_word2vec(sentences):
    """
    训练word2vec模型
    Args:
        sentences: 分词后的句子列表
    Returns:
        model: 训练好的word2vec模型
    """
    print("开始训练word2vec模型...")
    model = Word2Vec(
        sentences=sentences,
        vector_size=100,  # 词向量维度
        window=5,         # 窗口大小
        min_count=5,      # 最小词频
        workers=4,        # 使用4个CPU核心
        epochs=10         # 训练轮数
    )
    
    # 保存模型
    model.save('jinyong_word2vec.model')
    print("模型训练完成并已保存")
    return model

In [6]:
def extract_and_visualize_vectors(model, character_list):
    """
    提取人物词向量并进行可视化
    Args:
        model: 训练好的word2vec模型
        character_list: 人物列表
    """
    # 提取人物词向量
    character_vectors = {}
    for char in character_list:
        if char in model.wv:
            character_vectors[char] = model.wv[char]
    
    print(f"成功提取了 {len(character_vectors)} 个人物的词向量")
    
    # 转换为numpy数组
    vectors = np.array(list(character_vectors.values()))
    names = list(character_vectors.keys())
    
    # 进行PCA降维到2维
    pca_2d = PCA(n_components=2)
    vectors_2d = pca_2d.fit_transform(vectors)
    
    # 进行PCA降维到3维
    pca_3d = PCA(n_components=3)
    vectors_3d = pca_3d.fit_transform(vectors)
    
    # 2D可视化
    plt.figure(figsize=(12, 8))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
    for i, name in enumerate(names):
        plt.annotate(name, (vectors_2d[i, 0], vectors_2d[i, 1]))
    plt.title('金庸小说人物词向量2D可视化')
    plt.xlabel('第一主成分')
    plt.ylabel('第二主成分')
    plt.savefig('characters_2d.png')
    plt.close()
    
    # 3D可视化
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(vectors_3d[:, 0], vectors_3d[:, 1], vectors_3d[:, 2])
    for i, name in enumerate(names):
        ax.text(vectors_3d[i, 0], vectors_3d[i, 1], vectors_3d[i, 2], name)
    ax.set_title('金庸小说人物词向量3D可视化')
    ax.set_xlabel('第一主成分')
    ax.set_ylabel('第二主成分')
    ax.set_zlabel('第三主成分')
    plt.savefig('characters_3d.png')
    plt.close()
    
    print("可视化完成，结果已保存为characters_2d.png和characters_3d.png")

In [11]:
# 加载人物列表
character_list = load_character_list('characters_names.txt')

已加载 139 个人物


In [12]:
# 添加人物到jieba词典
add_characters_to_jieba(character_list)

已添加人物名字到jieba词典


In [13]:
# 读取并处理小说
sentences = read_and_tokenize_novels('novels')

正在处理文件: 金庸01飞狐外传.txt
正在处理文件: 金庸02雪山飞狐.txt
正在处理文件: 金庸03连城诀.txt
正在处理文件: 金庸04天龙八部.txt
正在处理文件: 金庸05射雕英雄传.txt
正在处理文件: 金庸06白马啸西风.txt
正在处理文件: 金庸07鹿鼎记.txt
正在处理文件: 金庸08笑傲江湖.txt
正在处理文件: 金庸09书剑恩仇录.txt
正在处理文件: 金庸10神雕侠侣.txt
正在处理文件: 金庸11侠客行.txt
正在处理文件: 金庸12倚天屠龙记.txt
正在处理文件: 金庸13碧血剑.txt
正在处理文件: 金庸14鸳鸯刀.txt
正在处理文件: 金庸15越女剑.txt
处理完成，共处理了 47441 个句子


In [14]:
# 训练模型
model = train_word2vec(sentences)  

开始训练word2vec模型...
模型训练完成并已保存


In [17]:
# 提取词向量并可视化
extract_and_visualize_vectors(model, character_list)

成功提取了 129 个人物的词向量
可视化完成，结果已保存为characters_2d.png和characters_3d.png
